In [11]:
import pathlib as pl
import pandas as pd
import collections as col

MOUNT_PREFIX = pl.Path("/mounts/hilbert/project")

REMOTE_PREFIX = "/gpfs/project"

data_path = pl.Path("projects/medbioinf/data/00_RESTRUCTURE/project-centric/platinum_pedigree/2024-01/assemblies/males")

sample_infos = col.defaultdict(dict)

def make_remote(file_path):
    
    file_path = str(file_path)
    file_path = file_path.replace(str(MOUNT_PREFIX), REMOTE_PREFIX)
    return file_path


for fasta_file in MOUNT_PREFIX.joinpath(data_path).glob("**/*.fasta"):
    filename = fasta_file.name
    sample_id = fasta_file.parents[2].name
    if "hap1" in filename or "haplotype1" in filename:
        asm_unit = "hap1"
    elif "hap2" in filename or "haplotype2" in filename:
        asm_unit = "hap2"
    else:
        print(f"skipping over {fasta_file}")
        continue
    sample_infos[sample_id][asm_unit] = make_remote(fasta_file)
    
sample_sheet = [(k, d["hap1"], d["hap2"]) for k, d in sample_infos.items()]

sample_sheet = pd.DataFrame.from_records(
    sample_sheet,
    columns=["sample", "asm_hap1", "asm_hap2"]
)
sample_sheet.sort_values("sample", inplace=True)

out_tsv = pl.Path("../../samples/platped_all.tsv").resolve()
print(out_tsv)

with open(out_tsv, "w") as dump:
    _ = dump.write("# 20240125T1559\n")
    sample_sheet.to_csv(dump, sep="\t", header=True, index=False)





skipping over /mounts/hilbert/project/projects/medbioinf/data/00_RESTRUCTURE/project-centric/platinum_pedigree/2024-01/assemblies/males/NA12889/verkko/1.3.1/assembly.fasta
/home/ebertp/work/code/cubi/project-run-hgsvc-hybrid-assemblies/samples/platped_all.tsv
